In [142]:
import requests
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt

In [2]:
import numpy as np
#import pandas as pd
#import re
import os
import codecs
#from sklearn import feature_extraction
import mpld3

In [59]:
#Doc2Vec/Word2vec packages
import nltk
from nltk.tokenize import sent_tokenize
import gensim
from gensim import corpora, models, similarities
import logging
import os
import wikipedia
import random
import json

In [ ]:
class mywikipedia(wikipedia.WikipediaPage):
    def __load(self, redirect=True, preload=False):


###Loop through many wikipedia pages
Runs them through training model
https://en.wikipedia.org/wiki/Special:ApiSandbox#action=query&prop=info&format=json&inprop=url&pageids=1000005

In [167]:
i = 0
counter = 0
good_id_list = []
while counter < 50:
    if i == 1000:
        print 'went on too long'
        break
    useid = 1000000 + i
    try:
        r = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info\
                &format=json&inprop=url&pageids=' + str(useid))
        rtext = r.text
        jsontext = json.loads(rtext)
        if jsontext['query']['pages'][str(useid)]['ns'] == 0:
            article = requests.get('https://en.wikipedia.org/?curid=' + str(useid))
            soup = BeautifulSoup(article.text, 'html.parser')
            paragraphs = soup.findAll('p')
            if len(paragraphs) >= 10:
                good_id_list.append(useid)
                counter += 1
        #print wikipedia.WikipediaPage(pageid=useid)
        i += 1
    except:
        i += 1
    if i % 100 == 0:
        print i
print good_id_list

100
200
[1000003, 1000007, 1000021, 1000025, 1000027, 1000028, 1000032, 1000034, 1000035, 1000037, 1000041, 1000042, 1000048, 1000053, 1000055, 1000060, 1000065, 1000069, 1000073, 1000081, 1000102, 1000116, 1000117, 1000118, 1000127, 1000128, 1000132, 1000135, 1000136, 1000142, 1000144, 1000146, 1000149, 1000155, 1000158, 1000159, 1000162, 1000165, 1000168, 1000181, 1000188, 1000193, 1000202, 1000208, 1000214, 1000226, 1000228, 1000232, 1000245, 1000252]


In [166]:
useid = 1000076
#article = requests.get('https://en.wikipedia.org/?curid=1000076')# + str(useid))
article = requests.get('https://en.wikipedia.org/wiki/William_Cleland')
soup = BeautifulSoup(article.text, 'html.parser')
paragraphs = soup.findAll('p')
len(paragraphs)

5

In [130]:
#r = requests.get('https://en.wikipedia.org/w/api.php?action=query&meta=siteinfo&\
#                 format=json&siprop=namespaces&pageids=1200008')
r = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info\
                &format=json&inprop=url&pageids=1000001')#   + str(useid))

In [133]:
rtext = r.text
jsontext = json.loads(rtext)
print jsontext['query']['pages']['1000001']['ns']
jsontext

0


{u'batchcomplete': u'',
 u'query': {u'pages': {u'1000001': {u'ns': 0,
    u'pageid': 1000001,
    u'title': u'NPU'}}},
 u'warnings': {u'main': {u'*': u"Unrecognized parameter: 'inprop'"},
  u'query': {u'*': u"Unrecognized value for parameter 'prop': info                "}}}

In [170]:
url = ['https://en.wikipedia.org/wiki/Myocardial_infarction',
      'https://en.wikipedia.org/wiki/Blood_flow',
      'https://en.wikipedia.org/wiki/Circulatory_system#Human_cardiovascular_system']
TaggedDocument = gensim.models.doc2vec.TaggedDocument
for u in url:
    response = requests.get(u)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    para = page_read(soup)
    print para
    print len(para)
    para = cleanText(para)
    para = labelizeReviews(para, 'Testing')
    model = gensim.models.Doc2Vec(para, size=200, window=8, min_count=5, workers=4)
    print model
    print model.estimate_memory()

Myocardial infarction (MI) or acute myocardial infarction (AMI), commonly known as a heart attack, occurs when blood flow stops to a part of the heart causing damage to the heart muscle. The most common symptom is chest pain or discomfort which may travel into the shoulder, arm, back, neck, or jaw. Often it is in the center or left side of the chest and lasts for more than a few minutes. The discomfort may occasionally feel like heartburn. Other symptoms may include shortness of breath, nausea, feeling faint, a cold sweat, or feeling tired.[1] About 30% of people have atypical symptoms,[2] with women more likely than men to present atypically.[3] Among those over 75 years old, about 5% have had an MI with little or no history of symptoms.[4] An MI may cause heart failure, an irregular heartbeat, or cardiac arrest.[5][6]
831
Doc2Vec(dm/s,d200,hs,w8,mc5,t4)
{'vocab': 17500, 'doctag_lookup': 166200, 'doctag_syn0': 664800, 'total': 888500, 'syn0': 20000, 'syn1': 20000}
Blood flow is the co

In [169]:
def page_read(soup):
    paragraphs = soup.findAll('p')
    theindex = 0
    wikidef = paragraphs[theindex].text
    return wikidef

In [147]:
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n','') for z in corpus]

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

In [148]:
def labelizeReviews(reviews, label_type):
    labelized = []
    for i,v in enumerate(reviews):
        label = '%s_%s'%(label_type,i)
        labelized.append(TaggedDocument(v, [label]))
    return labelized